In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%autoreload 2
from utils import *
from pymongo import MongoClient
from config import *
from tqdm import tqdm
from datetime import datetime
import db_utils
from bson.objectid import ObjectId

In [3]:
client = MongoClient(f"mongodb+srv://{DB_USER}:{DB_PASSWORD}@{DB_URL}/?retryWrites=true&w=majority")

db = client[DB_NAME]

db_utils.init(db)

collection rooms already exists
collection teachers already exists
collection courses already exists
collection teach_in already exists
collection course_bookings already exists
collection course_schedules already exists
collection studyplans already exists
collection etu_units already exists
collection semesters already exists
collection planned_in already exists
collection event_bookings already exists
collection event_schedules already exists
collection roles already exists
collection users already exists
collection units already exists
collection managed_by already exists


In [11]:
# List all courses url
courses_url = getAllCoursesUrl()

# Filter unique
courses_url = list(set(courses_url))

100%|██████████| 1/1 [00:09<00:00,  9.01s/it]


In [12]:
# Parse all courses
URL_ROOT = 'https://edu.epfl.ch'
courses = []
for url in tqdm(courses_url, total=len(courses_url)):
    course = parseCourse(URL_ROOT + url)
    courses.append(course)

# Filter duplicates
courses = filter_duplicates_courses(courses)

100%|██████████| 447/447 [04:19<00:00,  1.73it/s]


In [6]:
# Fall 2023-2024
create_new_semester(db,
    name="Semestre d'automne 2023-2024",
    start_date=datetime(2023, 9, 19),
    end_date=datetime(2023, 12, 22),
    type="fall",
    available=True
)

# Spring 2023-2024
create_new_semester(db,
    name="Semestre de printemps 2023-2024",
    start_date=datetime(2024, 2, 19),
    end_date=datetime(2024, 5, 31),
    type="spring",
    available=True
)

In [7]:
### -- Create all courses -- ###
create_courses(db, courses)

100%|██████████| 1869/1869 [00:00<00:00, 413217.76it/s]


In [17]:
### -- Create all rooms -- ###
create_rooms(db, courses)

Getting rooms from plan.epfl.ch


100%|██████████| 9/9 [00:10<00:00,  1.16s/it]


Getting rooms from database
Updating rooms in database


100%|██████████| 168/168 [00:00<00:00, 1706.19it/s]


Creating new rooms in database


100%|██████████| 1/1 [00:00<00:00, 887.50it/s]


In [9]:
### -- Create all teachers -- ###
create_teachers(db, courses)

100%|██████████| 1869/1869 [00:00<00:00, 36711.85it/s]


In [10]:
### -- Create all teach_in -- ###
create_teach_in(db, courses)

100%|██████████| 1869/1869 [02:48<00:00, 11.12it/s]


In [18]:
### -- Create all bookings and schedules -- ###
create_courses_schedules_and_bookings(db, courses)

100%|██████████| 447/447 [01:17<00:00,  5.74it/s]


In [11]:
### -- List planned_in -- ###
planned_in = list_courses_plan_studyplans()

In [14]:
### -- Create all units -- ###
create_units(db, planned_in)

100%|██████████| 174/174 [00:00<00:00, 528847.03it/s]


In [15]:
### -- Create all studyplans -- ###
create_studyplans(db, planned_in)

100%|██████████| 2849/2849 [00:00<00:00, 303704.88it/s]


In [16]:
### -- Create all planned_in -- ###
create_planned_in(db, planned_in)

100%|██████████| 2849/2849 [01:34<00:00, 30.02it/s]
